# Кэширование в Python с использованием алгоритма кэширования LRU

Данная публикация – незначительно сокращенный перевод статьи Сантьяго Валдаррама [Caching in Python Using the LRU Cache Strategy](https://realpython.com/lru-cache-python/).

Кэширование – один из подходов, который при правильном использовании значительно ускоряет работу и снижает нагрузку на вычислительные ресурсы. В модуле стандартной библиотеки Python `functools` реализован декоратор `@lru_cache`, дающий возможность кэшировать вывод функций, используя стратегию [Least Recently Used](https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D1%8B_%D0%BA%D1%8D%D1%88%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F) (LRU, «вытеснение давно неиспользуемых»). Это простой, но мощный метод, который позволяет использовать в коде возможности кэширования.

В этом руководстве мы рассмотрим:
- Какие стратегии кэширования доступны и как их реализовать с помощью декораторов
- что такое LRU и как работает этот подход;
- как повысить производительность за счет кэширования с помощью декоратора `@lru_cache`;
- как расширить функциональность декоратора `@lru_cache` и прекратить его работу по истечении определенного интервала времени.

К концу этого руководства у вас будет более глубокое понимание того, как работает кэширование и как использовать его преимущества в Python.


# Кэширование и в чем его польза
**Кэширование** – это метод оптимизации хранения недавних или часто используемых данных в тех местах памяти, которые быстрее или дешевле с точки зрения вычислений, чем их источник.

Представьте, что вы создаете приложение для чтения новостей, которое агрегирует новости из различных источников. Пользователь перемещается по списку, приложение загружает статьи и отображает их на экране.

Что произойдет, если пользователь решит многократно перемещаться между парой новостных статей? Без кэширования приложению придется каждый раз получать одно и то же содержимое. В этом случае неэффективно используется и система пользователя, и наш сервер со статьями, на котором создается дополнительная нагрузка.

Лучшим подходом после получения каждой статьи было бы хранить контент локально. Когда пользователь в следующий раз откроет статью, приложение сможет открыть контент из локально сохраненной копии, вместо того, чтобы загово загружать его из источника. В информатике этот метод называется *кэшированием*.

# Реализация кэширования посредством словаря Python
Можно реализовать решение для кэширования в Python, используя словарь. Касательного примера: вместо того, чтобы каждый раз обращаться непосредственно к серверу, мы можем проверить, есть ли у нас контент в кэше, и вернуться на сервер только в том случае, если его нет. Мы можем использовать URL статьи в качестве ключа, а ее содержимое – в качестве значения:

In [2]:
import requests

cache = dict()

def get_article_from_server(url):
    print("Забираем статью с сервера...")
    response = requests.get(url)
    return response.text

def get_article(url):
    print("Получаем статью...")
    if url not in cache:
        cache[url] = get_article_from_server(url)

    return cache[url]

get_article("https://realpython.com/sorting-algorithms-python/")[:1000]
get_article("https://realpython.com/sorting-algorithms-python/")[:1000]

Получаем статью...
Забираем статью с сервера...
Получаем статью...


'\n<!doctype html>\n<html lang="en">\n<head>\n<link href="https://files.realpython.com" rel="preconnect">\n<title>Sorting Algorithms in Python – Real Python</title>\n<meta name="author" content="Real Python">\n<meta name="description" content="In this tutorial, you&#x27;ll learn all about five different sorting algorithms in Python from both a theoretical and a practical standpoint. You&#x27;ll also learn several related and important concepts, including Big O notation and recursion.">\n<meta name="keywords" content="">\n<meta charset="utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n<link rel="stylesheet" href="/static/gfonts/font.32be62914940.css">\n<link rel="stylesheet" href="/static/realpython.min.5b53688b6129.css">\n<link rel="canonical" href="https://realpython.com/sorting-algorithms-python/">\n<meta name="twitter:card" content="summary_large_image">\n<meta name="twitter:image" content="https://files.realpython.com/media/Sorting-Al

**Примечание**. Для выполнение этого примера у вас должна быть установлена библиотека requests.

In [3]:
#раскомментируйте следующую строку,
# чтобы установить библиотеку из блокнота Jupyter
#!pip install requests

Обратите внимание на то, что несмотря на двойной вызов `get_article()`, статья с сервера загружается лишь один раз. Это происходит потому, что после первого доступа к статье мы помещаем ее URL и содержимое в словарь `cache`. Во второй раз код не требует повторного получения элемента с сервера.


# Стратегии кэширования
Однако в этой простой реализации кэширования есть проблема: содержимое словаря будет неограниченно расти. По мере того, как пользователь загружает больше статей, приложение будет сохранять их в памяти, и в конечном итоге может произойти сбой приложения.

Чтобы обойти эту проблему, нам нужна стратегия, которая позволит программе решить, какие статьи должны остаться в памяти, а какие – пора удалить. Эти стратегии кэширования представляют собой алгоритмы, которые фокусируются на управлении кэшированной информацией и выборе элементов, которые следует удалить, чтобы освободить место для новых.

Существует несколько различных стратегий, которые можно использовать для удаления элементов из кэша и предотвращения превышения его максимального размера. Пять самых популярных перечислены в таблице.

Стратегия | Какую запись удаляем | Эти записи чаще других используют повторно
--- | --- | ---
First-In/First-Out (FIFO) | Самая старая | Новые
Last-In/First-Out (LIFO) | Самая недавняя | Старые
Least Recently Used (LRU) | Использовалась наиболее давно | Использовилась недавно
Most Recently Used (MRU) | Использовалась последней | Использовались наиболее давно 
Least Frequently Used (LFU) | Использовалась наиболее редко | Использовались часто


# Погружаемся в идею LRU-кэширования
Кэш, реализованный посредством стратегии LRU, упорядочивает элементы в порядке их использования. Каждый раз, когда мы обращаемся к записи, алгоритм LRU перемещает ее в верхнюю часть кэша. Таким образом, алгоритм может быстро определить запись, которая дольше всех не использовалась, проверив конец списка.

На следующем рисунке показано представление кэша после того, как пользователь запросил статью из сети.

<img src="https://files.realpython.com/media/lru_cache_1_1.2eb80a8b24a3.png" width="500"/>

Кэш сохраняет статью в последнем слоте перед тем, как передать ее пользователю. На следующем рисунке показано, что происходит, когда пользователь запрашивает вторую статью.

<img src="https://files.realpython.com/media/lru_cache_2_1.8c4f225e79d0.png" width="500"/>

Вторая статья занимает самый последний слот, перемещая первую статью вниз по списку.

Стратегия LRU предполагает, что чем позже использовался объект, тем больше вероятность, что он понадобится в будущем, поэтому алгоритма сохраняет этот объект в кэше в течение максимально длительного времени.

# Заглядываем за кулисы кэша LRU

Один из способов реализовать кэш LRU в Python – использовать комбинацию [двусвязного списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%8F%D0%B7%D0%BD%D1%8B%D0%B9_%D1%81%D0%BF%D0%B8%D1%81%D0%BE%D0%BA) и [хеш-таблицы](https://ru.wikipedia.org/wiki/%D0%A5%D0%B5%D1%88-%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0). Головной элемент двусвязного списка указывает на последнюю запрошенную запись, а хвостовой – на наиболее давно использовавшуюся запись.

На рисунке ниже показана возможная структура реализации кэша LRU.

<img src="https://files.realpython.com/media/cache_internal_representation_1.6fdd3a39fa28.png" width="400"/>

Используя хеш-таблицу, мы обеспечиваем доступ к каждому элементу в кэше, сопоставляя каждую запись с определенным местом в двусвязном списке. При этом доступ к наименее недавно использовавшемуся элементу и обновление кэша – это операции, выполняемые за константное время (то есть с [временной сложностью](https://ru.wikipedia.org/wiki/%D0%92%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%BB%D0%BE%D0%B6%D0%BD%D0%BE%D1%81%D1%82%D1%8C_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D0%B0) алгоритма  $O(1)$).

---
**Примечание**. Примеры временной сложности различных функций Python рассматривались на proglib в статье [«Сложность алгоритмов и операций на примере Python»](https://proglib.io/p/slozhnost-algoritmov-i-operaciy-na-primere-python-2020-11-03).

---

Начиная с версии 3.2, для реализации стратегии LRU Python включает декоратор `@lru_cache`.


# Использование `@lru_cache` для реализации кэша LRU в Python

Как и решение для кэширования, которое мы реализовали ранее, декоратор `@lru_cache` за кулисами использует словарь. Результат выполнения функции кэшируется под ключом, соответствующим вызову функции и предоставленным аргументам. Это означает, что аргументы должны быть хешируемыми, чтобы декоратор работал.


## Наглядное представление алгоритма: перепрыгиваем ступеньки
Представим, что мы хотим определить все способы, которыми можем достичь определенной ступеньки на лестнице. Сколько есть способов, например, добраться до четвертой ступеньки, если мы можем переступить-перепрыгнуть 1, 2, 3 (но не более) ступеньки? На рисунке ниже представлены эти различные комбинации.

<img src="https://files.realpython.com/media/7_ways_to_make_it_to_4th_stair.2dea6527a4b7.png" width="500"/>

Под каждым из рисунков приведен путь с указанием числа ступенек, преодоленных за один прыжок. При этом количество способов достижения четвертой ступеньки равно общему числу способов, которыми можно добраться до третьей, второй и первой ступеньки:

<img src="https://files.realpython.com/media/combination-jumps-to-7th-stair-lru-cache_1.385e09435b81.png" width="500"/>

Получается, что решение первой задачи можно разложить на более мелкие подзадачи. Чтобы определить различные пути к четвертой ступеньке, мы можем сложить четыре способа достижения третьей ступеньки, два способа достижения второй ступеньки и один способ достижения первой. То есть можно использовать рекурсивный подход.

Опишем программно рекурсивное решение в лоб, в точности, как мы его сейчас видим:

In [4]:
def steps_to(stair):
    if stair == 1:
        # До первой ступеньки можно добраться с пола 
        # единственным образом
        return 1
    elif stair == 2:
        # Второй ступеньки можно достингуть,
        # ступая по одной за раз или преодолев сразу две
        return 2
    elif stair == 3:
        # Чтобы добраться до третьей ступеньки:
        # 1. Перепрыгнуть сразу до третьей
        # 2. Перепрыгнуть две, потом одну
        # 3. Перепрыгнуть одну потом две
        # 4. По одной за раз
        return 4
    else:
        # Все промежуточные шаги это различные
        # варианты прыжков через 1, 2 или 3 ступеньки,
        # так что общее число вариантов - это сумма
        # таких комбинаций
        return (
            steps_to(stair - 3)
            + steps_to(stair - 2)
            + steps_to(stair - 1)
        )

print(steps_to(4))

7


Код работает для 4 ступенек. Давайте проверим, как он подсчитает число вариантов для лестницы из 30 ступенек.

In [5]:
steps_to(30)

53798080

Получилось свыше 53 млн. комбинаций. Однако когда мы искали решение для тридцатой ступеньки, сценарий мог длиться довольно долго. Интересно, как быстро работает наш код.

# Засекаем время выполнения программного кода
Измерим, сколько времени занимает выполнение написанного нами кода.

---
**Примечание**. О различных вариантах работы со временем в Python вы можете прочитать в публикации [«Назад в будущее: практическое руководство по путешествию во времени с Python»](https://proglib.io/p/nazad-v-budushchee-prakticheskoe-rukovodstvo-po-puteshestviyu-vo-vremeni-s-python-2019-12-01). Эта публикация также адаптирована в виде [блокнота Jupyter](https://github.com/matyushkin/lessons/blob/master/time/time_related.ipynb).

---

Для этого мы можем использовать модуль Python `timeit` или соответствующую команду в блокноте Jupyter.

In [6]:
%%timeit
steps_to(30)

3.31 s ± 45.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Количество секунд, которое вы увидите, зависит от характеристик используемого компьютера. В моей системе расчет занял 3 секунды, что довольно медленно для всего тридцати ступенек. Это решение может быть значительно улучшено c помощью [мемоизации](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D0%BC%D0%BE%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F).

---
**Примечание**. Один из примеров мемоизации рассматривался в статье [«Python и динамическое программирование на примере задачи о рюкзаке»](https://proglib.io/p/python-i-dinamicheskoe-programmirovanie-na-primere-zadachi-o-ryukzake-2020-02-04).

---

# Использование мемоизации для улучшения решения

Наша рекурсивная реализация решает проблему, разбивая ее на более мелкие шаги, которые дополняют друг друга. На следующем рисунке показано дерево для семи ступенек, в котором каждый узел представляет определенный вызов `steps_to()`:

<img src="https://robocrop.realpython.net/?url=https%3A//files.realpython.com/media/tree.3ebda400089a.png&w=1111&sig=849b7efff92dde86fb1d52e44f1f4e7aec27e3cc" width="500"/>

Можно заметить, что алгоритму приходится вызывать `steps_to()` с одним и тем же аргументом несколько раз. Например, `steps_to(5)` вычисляется два раза, `steps_to(4)` – четыре раза, `steps_to(3)` – семь раз и т. д.. Вызов одной и той же функции несколько раз запускает вычисления, в которых нет необходимости – результат всегда будет одним и тем же.

Чтобы решить эту проблему, мы можем использовать метод, называемый мемоизацией: мы запоминаем результат, полученный для одних и тех же входных значений и затем возвращаем их при следующем аналогичном запросе. Звучит как прекрасная возможность использовать декоратор `@lru_cache`!

---
**Примечание**. Если вы незнакомы с концепцией декораторов, но хотите глубже разобраться в этом вопросе, просто прочитайте статью [Всё, что нужно знать о декораторах Python](https://proglib.io/p/vse-chto-nuzhno-znat-o-dekoratorah-python-2020-05-09) (она также адаптирована в формате [Jupyter](https://github.com/matyushkin/lessons/blob/master/decorators/decorators.ipynb) и [Colab](https://colab.research.google.com/github/matyushkin/lessons/blob/master/decorators/decorators.ipynb)). Для наших задач достаточно знать, что это функции-обертки, которые позволяют модифицировать поведение других функций и классов. Чтобы применить декоратор, достаточно объявить его перед определением функции.

---


Импортируем декоратор из модуля `functools` и применим к основной функции.

In [7]:
from functools import lru_cache

@lru_cache()
def steps_to(stair):
    if stair == 1:
        return 1
    elif stair == 2:
        return 2
    elif stair == 3:
        return 4
    else:
        return (steps_to(stair - 3)
                + steps_to(stair - 2)
                + steps_to(stair - 1))

**Примечание**.  В Python 3.8 и выше,  если вы не указываете никаких параметров, можно использовать декоратор `@lru_cache` без скобок. В более ранних версиях необходимо добавить круглые скобки: `@lru_cache()`.

In [8]:
%%timeit
steps_to(30)

60.6 ns ± 3.58 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


От единиц секунд к десяткам наносекунд – потрясающее улучшение, обязанное тем, что за кулисами декоратор `@lru_cache` сохраняет результаты вызова `steps_to()` для каждого отдельного ввода. Каждый раз, когда код вызывает функцию с теми же параметрами, вместо того, чтобы заново вычислять ответ, он возвращает правильный результат прямо из памяти.


# Другие возможности @lru_cache
Подключив декоратор `@lru_cache`, мы сохраняем каждый вызов и ответ в памяти для последующего доступа, если они потребуется снова. Но сколько таких комбинаций мы можем сохранить, пока не иссякнет память?

У декоратора `@lru_cache` есть атрибут `maxsize`, который определяет максимальное количество записей до того, как кэш начнет удалять старые элементы. По умолчанию `maxsize` равен 128. Если мы присвоим `maxsize` значение `None`, то кэш будет расти без всякого удаления записей. Это может стать проблемой, если мы храним в памяти слишком большое количество различных вызовов.

Применим `@lru_cache` с использованием атрибута `maxsize` и добавим вызов метода `cache_info()`:

In [9]:
@lru_cache(maxsize=16)
def steps_to(stair):
    if stair == 1:
        return 1
    elif stair == 2:
        return 2
    elif stair == 3:
        return 4
    else:
        return (steps_to(stair - 3)
                + steps_to(stair - 2)
                + steps_to(stair - 1))
    
steps_to(30)
print(steps_to.cache_info())

CacheInfo(hits=52, misses=30, maxsize=16, currsize=16)


Мы можем использовать информацию, возвращаемую `cache_info()`, чтобы понять, как работает кэш, и настроить его, чтобы найти подходящий баланс между скоростью работы и объемом памяти:

- `hits=52` – количество вызовов, которые  `@lru_cache` вернул непосредственно из памяти, поскольку они присутствовали в кэше;
- `misses=30` – количество вызовов, которые взяты не из памяти, а были вычислены (в случае нашей задачи это каждая новая ступень);
- `maxsize=16` – это размер кэша, который мы определили, передав его  декоратору;
- `currsize=16` – текущий размер кэша, в этом случае кэш заполнен.


# Добавление срока действия кэша
Перейдем от наглядного игрушечного примера к более реалистичному. Представьте, что мы хотим отслеживать появление на ресурсе Real Python новых статей, содержащих в заголовке слово `python` – выводить название, скачивать статью и отображать ее объем (число символов).

Real Python предоставляет [протокол Atom](https://ru.wikipedia.org/wiki/Atom), так что мы можем использовать библиотеку `feedparser` для анализа канала и библиотеку `requests` для загрузки содержимого статьи, как мы это делали раньше.

In [10]:
# расскоментируйте строку, чтобы установить библиотеку feedparser
#!pip install feedparser

In [11]:
import feedparser
import requests
import ssl
import time

if hasattr(ssl, "_create_unverified_context"):
    ssl._create_default_https_context = ssl._create_unverified_context

def get_article_from_server(url):
    print("Получение статьи с сервера...")
    response = requests.get(url)
    return response.text

def monitor(url):
    maxlen = 45
    while True:
        try:
            print("\nПроверям ленту...")
            feed = feedparser.parse(url)

            for entry in feed.entries[:5]:
                if "python" in entry.title.lower():
                    truncated_title = (
                        entry.title[:maxlen] + "..."
                        if len(entry.title) > maxlen
                        else entry.title
                    )
                    print(
                        "Совпадение:",
                        truncated_title,
                        len(get_article_from_server(entry.link)),
                    )

            time.sleep(5)
        except KeyboardInterrupt:
            break

Скрипт будет работать непрерывно, пока мы не остановим его, нажав `Ctrl + C` в окне терминала (или не прервем выполнение в Jupyter-блокноте).

In [12]:
monitor("https://realpython.com/atom.xml")


Проверям ленту...
Получение статьи с сервера...
Найдено совпадение: The Real Python Podcast – Episode #35: Securi... 28704
Получение статьи с сервера...
Найдено совпадение: PyPy: Faster Python With Minimal Effort 67389
Получение статьи с сервера...
Найдено совпадение: Handling Missing Keys With the Python default... 33224
Получение статьи с сервера...
Найдено совпадение: Use Sentiment Analysis With Python to Classif... 158400
Получение статьи с сервера...
Найдено совпадение: The Real Python Podcast – Episode #34: The Py... 29576

Проверям ленту...
Получение статьи с сервера...
Найдено совпадение: The Real Python Podcast – Episode #35: Securi... 28704
Получение статьи с сервера...
Найдено совпадение: PyPy: Faster Python With Minimal Effort 67389
Получение статьи с сервера...
Найдено совпадение: Handling Missing Keys With the Python default... 33224
Получение статьи с сервера...
Найдено совпадение: Use Sentiment Analysis With Python to Classif... 158401
Получение статьи с сервера...
Най

Используя `feedparser`, код загружает и анализирует xml-файл из RealPython.
Далее цикл перебирает первые пять записей в списке. Если слово `python` является частью заголовка, код печатает заголовок и длину статьи. Далее код «засыпает» на 5 секунд, после чего вновь запускается мониторинг.

Каждый раз, когда сценарий загружает статью, в консоль выводится сообщение «Получение статьи с сервера...». Если мы позволим скрипту работать достаточно долго, мы увидим, что это сообщение появляется повторно даже при загрузке той же ссылки.

Это отличная возможность кэшировать содержание статьи и избежать загрузки из сети каждые пять секунд. Мы можем использовать декоратор `@lru_cache`, однако содержание статьи со временем может измениться.

При первой загрузке статьи декоратор сохранит ее содержимое и каждый раз будет возвращать одни и те же данные. Если сообщение обновлено, то сценарий мониторинга никогда не осознает этого, потому что будет извлекать из кэша старую копию. Чтобы решить эту проблему, мы должны установить срок хранения записей в кэше.


# Критерии исключения записей из кэша

Итак, чтобы обойти описанную проблему, мы должны обновить реализацию кеша, чтобы срок его действия через определенное время истек. Мы можем реализовать эту идею в новом декораторе, который расширяет `@lru_cache`. Кэш должен возвращать результат на запрос только, если срок кэширования записи еще не истек, в обратном случае результат должен забираться с сервера. Вот возможная реализация этого нового декоратора:

In [13]:
from functools import lru_cache, wraps
from datetime import datetime, timedelta

def timed_lru_cache(seconds: int, maxsize: int = 128):
    def wrapper_cache(func):
        func = lru_cache(maxsize=maxsize)(func)
        
        # инструментирование декоратора двумя атрибутами,
        # представляющими время жизни кэша lifetime
        # и дату истечения срока его действия expiration
        func.lifetime = timedelta(seconds=seconds)
        func.expiration = datetime.utcnow() + func.lifetime

        @wraps(func)
        def wrapped_func(*args, **kwargs):
            if datetime.utcnow() >= func.expiration:
                func.cache_clear()
                func.expiration = datetime.utcnow() + func.lifetime

            return func(*args, **kwargs)

        return wrapped_func

    return wrapper_cache

Декоратор `@timed_lru_cache` реализует функциональность для оперирования временем жизни записей в кэше (в секундах) и максимальным размером кэша.

Код оборачивает функцию декоратором `@lru_cache`. Это позволяет нам использовать уже знакомую функциональность кэширования. 

Перед доступом к записи в кэше декоратор проверяет, не наступила ли текущая дата истечения срока действия. Если это так, декоратор очищает кэш и повторно вычисляет время жизни и срок действия. Время жизни распространяется на кэш в целом, а не на отдельные статьи.


# Кэширование статей с помощью нового декоратора

Теперь мы можем использовать новый декоратор `@timed_lru_cache` с функцией `monitor()`, чтобы предотвратить скачивание с сервера содержимого статьи при каждом новом запросе. Собрав код в одном месте, получим следующий результат:

In [19]:
import feedparser
import requests
import ssl
import time

from functools import lru_cache, wraps
from datetime import datetime, timedelta

if hasattr(ssl, "_create_unverified_context"):
    ssl._create_default_https_context = ssl._create_unverified_context

def timed_lru_cache(seconds: int, maxsize: int = 128):
    def wrapper_cache(func):
        func = lru_cache(maxsize=maxsize)(func)
        func.lifetime = timedelta(seconds=seconds)
        func.expiration = datetime.utcnow() + func.lifetime

        @wraps(func)
        def wrapped_func(*args, **kwargs):
            if datetime.utcnow() >= func.expiration:
                func.cache_clear()
                func.expiration = datetime.utcnow() + func.lifetime

            return func(*args, **kwargs)

        return wrapped_func

    return wrapper_cache

@timed_lru_cache(60)
def get_article_from_server(url):
    print("Получение статьи с сервера...")
    response = requests.get(url)
    return response.text

def monitor(url):
    maxlen = 45
    while True:
        try:
            print("\nПроверяем ленту...")
            feed = feedparser.parse(url)

            for entry in feed.entries[:5]:
                if "python" in entry.title.lower():
                    truncated_title = (
                        entry.title[:maxlen] + "..."
                        if len(entry.title) > maxlen
                        else entry.title
                    )
                    print(
                        "Совпадение:",
                        truncated_title,
                        len(get_article_from_server(entry.link)),
                    )

            time.sleep(5)
        except KeyboardInterrupt:
            break
        
monitor("https://realpython.com/atom.xml")


Проверяем ленту...
Получение статьи с сервера...
Найдено совпадение: The Real Python Podcast – Episode #35: Securi... 28704
Получение статьи с сервера...
Найдено совпадение: PyPy: Faster Python With Minimal Effort 67389
Получение статьи с сервера...
Найдено совпадение: Handling Missing Keys With the Python default... 33224
Получение статьи с сервера...
Найдено совпадение: Use Sentiment Analysis With Python to Classif... 158400
Получение статьи с сервера...
Найдено совпадение: The Real Python Podcast – Episode #34: The Py... 29576

Проверяем ленту...
Найдено совпадение: The Real Python Podcast – Episode #35: Securi... 28704
Найдено совпадение: PyPy: Faster Python With Minimal Effort 67389
Найдено совпадение: Handling Missing Keys With the Python default... 33224
Найдено совпадение: Use Sentiment Analysis With Python to Classif... 158400
Найдено совпадение: The Real Python Podcast – Episode #34: The Py... 29576

Проверяем ленту...
Найдено совпадение: The Real Python Podcast – Episode #3

Обратите внимание, как код печатает сообщение «Получение статьи с сервера ...» при первом доступе к соответствующим статьям. После этого, в зависимости от скорости cети, сценарий будет извлекать статьи из кэша несколько раз, прежде чем снова обратиться к серверу.

В приведенном примере скрипт пытается получить доступ к статьям каждые 5 секунд, а срок действия кэша истекает раз в минуту.


# Заключение

